In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline

In [ ]:
df = pd.read_csv("../input/buyers-time-prediction-challenge/ParticipantData_BTPC/Train.csv")
test = pd.read_csv("../input/buyers-time-prediction-challenge/ParticipantData_BTPC/Test.csv")
sample = pd.read_csv("../input/buyers-time-prediction-challenge/ParticipantData_BTPC/Sample Submission.csv")

In [ ]:
df.head()

In [ ]:
test.head()

In [ ]:
sample.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.shape,test.shape

In [ ]:
df.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df['device_details'] = lb.fit_transform(df.device_details)

In [ ]:
df.head()

In [ ]:
df.drop('session_id',axis=1,inplace=True)
df.head()

In [ ]:
re = pd.DataFrame(df['client_agent'].unique())

In [ ]:
re.reset_index(inplace=True)
re.shape

In [ ]:
re.head()

In [ ]:
dict1 = pd.Series(re['index'].values, index = re[0]).to_dict()

In [ ]:
df['client_agent'] = df.client_agent.map(dict1)

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'],errors='coerce',format='%Y-%m-%d')

In [ ]:
df.info()

In [ ]:
df['date_year'] = df['date'].dt.year
df['date_month'] = df['date'].dt.month
df['date_day'] = df['date'].dt.day

In [ ]:
df.drop('date',axis = 1,inplace=True)

In [ ]:
corr_matrix = df.corr()
corr_matrix['time_spent'].sort_values(ascending = False)

In [ ]:
plt.figure(figsize=(12,7))
sns.heatmap(corr_matrix,annot=True)

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
correlation(df,0.8)

In [ ]:
df.drop('date_month',axis=1,inplace=True)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
X_temp = df.drop(['time_spent'],axis=1)
Y_temp = df['time_spent']
model = ExtraTreesRegressor()
model.fit(X_temp,Y_temp)
feat_import = pd.Series(model.feature_importances_,index = X_temp.columns)
feat_import.nlargest(13).plot(kind='bar')
plt.show()

In [ ]:
df.head()

In [ ]:
sns.distplot(df.time_spent)

In [ ]:
df.time_spent = np.log1p(df.time_spent)
sns.kdeplot(df.time_spent)

In [ ]:
sns.distplot(df.time_spent)

In [ ]:
X = df.drop('time_spent',axis=1)
Y = df['time_spent']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 42)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression,Lasso,ElasticNet,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error

In [ ]:
lr = LinearRegression()
lr.fit(X_train,Y_train)
pred = np.expm1(lr.predict(X_test))
s1 = np.sqrt(mean_squared_log_error(Y_test,pred))
s1

In [ ]:
ls = Lasso(alpha=1)
ls.fit(X_train,Y_train)
pred_ls = np.expm1(ls.predict(X_test))
s2 = np.sqrt(mean_squared_log_error(Y_test,pred_ls))
s2

In [ ]:
rd = Ridge(alpha=1)
rd.fit(X_train,Y_train)
pred_rd = np.expm1(rd.predict(X_test))
s3 = np.sqrt(mean_squared_log_error(Y_test,pred_rd))
s3

In [ ]:
en = ElasticNet(alpha=1)
en.fit(X_train,Y_train)
pred_en = np.expm1(en.predict(X_test))
s4 = np.sqrt(mean_squared_log_error(Y_test,pred_en))
s4

In [ ]:
dc = DecisionTreeRegressor()
dc.fit(X_train,Y_train)
pred_dc = np.expm1(dc.predict(X_test))
s5 = np.sqrt(mean_squared_log_error(Y_test,pred_dc))
s5

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train,Y_train)
pred_rf = np.expm1(rf.predict(X_test))
s6 = np.sqrt(mean_squared_log_error(Y_test,pred_rf))
s6

In [ ]:
cat = CatBoostRegressor(iterations=500,loss_function='MAE',eval_metric='RMSE',task_type='GPU')
cat.fit(X_train,Y_train,verbose=True)
pred_cat = np.expm1(cat.predict(X_test))
s7 = np.sqrt(mean_squared_log_error(Y_test,pred_cat))
s7

In [ ]:
params = {'n_estimators': 1000,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.0002,
          'loss': 'ls'}
gd =GradientBoostingRegressor(**params)
gd.fit(X_train,Y_train)
pred_gd = np.expm1(gd.predict(X_test))
s8 = np.sqrt(mean_squared_log_error(Y_test,pred_gd))
s8

In [ ]:
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(gd.staged_predict(X_test)):
    test_score[i] = gd.loss_(Y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, gd.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
plt.show()

In [ ]:
xg = XGBRegressor(n_estimators = 3000,learning_rate=0.01)
xg.fit(X_train,Y_train)
pred_xg = np.expm1(xg.predict(X_test))
s9 = np.sqrt(mean_squared_log_error(Y_test,pred_xg))
s9

In [ ]:
lgb = LGBMRegressor()
lgb.fit(X_train,Y_train,eval_set = (X_test,Y_test),early_stopping_rounds=1000)
pred_lgb = np.expm1(lgb.predict(X_test))
s10 = np.sqrt(mean_squared_log_error(Y_test,pred_lgb))
s10

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]



random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}




rf_random = RandomizedSearchCV(estimator=rf,param_distributions=random_grid,
                               scoring='neg_mean_squared_error',
                              n_iter=10,cv=5,verbose=2,random_state=42,n_jobs=1)

rf_random.fit(X_train,Y_train)

In [ ]:
rf_random.best_params_

In [ ]:
predictions = np.expm1(rf_random.predict(X_test))
s11 = np.sqrt(mean_squared_log_error(Y_test,predictions))
s11

In [ ]:
svr = SVR()
svr.fit(X_train,Y_train)
pred_svr = np.expm1(svr.predict(X_test))
s12 = np.sqrt(mean_squared_log_error(Y_test,pred_svr))
s12

In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_train,Y_train)
pred_knn = np.expm1(knn.predict(X_test))
s13 = np.sqrt(mean_squared_log_error(Y_test,pred_knn))
s13

In [ ]:
test = pd.read_csv('../input/buyers-time-prediction-challenge/ParticipantData_BTPC/Test.csv')

In [ ]:
test.head()

In [ ]:
test.drop('session_id',axis=1,inplace=True)

In [ ]:
test['client_agent'] = test.client_agent.map(dict1)

In [ ]:
test['device_details'] = lb.fit_transform(test.device_details)

In [ ]:
test['date'] = pd.to_datetime(test['date'],errors='coerce',format='%Y-%m-%d')
test['date_year'] = test['date'].dt.year
test['date_month'] = test['date'].dt.month
test['date_day'] = test['date'].dt.day

In [ ]:
test.drop('date_month',axis =1,inplace = True)

In [ ]:
test.drop('date',axis=1,inplace=True)

In [ ]:
test.client_agent = test.client_agent.fillna(test.client_agent.mode().iloc[0])

In [ ]:
test = sc.transform(test)

In [ ]:
# test_ridge = np.expm1(rd.predict(test))
# test_lr = np.expm1(lr.predict(test))
# final_test = (test_ridge*0.6 + test_lr*0.4)

In [ ]:
from mlxtend.regressor import StackingCVRegressor
stack = StackingCVRegressor(regressors=(XGBRegressor(), 
                            Ridge(), LinearRegression(),LGBMRegressor(),RandomForestRegressor(n_estimators=300,min_samples_split=15,min_samples_leaf=1,max_features='sqrt',max_depth=15),CatBoostRegressor(),GradientBoostingRegressor()),
                            meta_regressor=Lasso(), cv=10,
                            use_features_in_secondary=True,
                            store_train_meta_features=True,
                            shuffle=False,
                            random_state=1)

In [ ]:
stack.fit(X_train,Y_train)
final_test = np.expm1(stack.predict(test))

In [ ]:
# test_xg = np.expm1(xg.predict(test))
# test_lgb = np.expm1(lgb.predict(test))
# test_cat = np.expm1(cat.predict(test))
# test_rf_random = np.expm1(rf_random.predict(test))
# final_test = (test_xg*0.6 + test_lgb*0.2 + test_cat*0.1 + test_rf_random*0.1 )

In [ ]:
# test_ridge = np.expm1(rd.predict(test))
# test_lr = np.expm1(lr.predict(test))
# test_rf_random = np.expm1(rf_random.predict(test))
# test_lgb = np.expm1(lgb.predict(test))
# final_test = (test_ridge*0.6 + test_lr*0.20 + test_rf_random*0.1 + test_lgb*0.1)

In [ ]:
sample['time_spent'] = final_test

In [ ]:
sample.head()

In [ ]:
sample.to_csv('submit_rd_lr.csv',index=False)